In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

plt.rcParams["figure.figsize"] = [5, 3.1]

# Today: Exact solutions and error

Tom Ranner

# Recap

### Differential equation

$$
y'(t) = f(t, y(t)) \quad \text{subject to the initial condition} \quad y(t_0) = y_0.
$$

$f$ is called the right-hand side.

### Euler's method

1. Set initial values for $t^{(0)}$ and $y^{(0)}$
2. Loop over all time steps, until the final time, updaing using the formulae:

   $$
   \begin{aligned}
   y^{(i+1)} & = y^{(i)} + \mathrm{d}t f(t^{(i)}, y^{(i)}) \\
   t^{(i+1)} & = t^{(i)} + \mathrm{d}t.
   \end{aligned}
   $$

# Exact derivatives and exact solutions

In *some* cases, it is possible to evaluate derivatives of a function exactly.

In *some* cases, we can solve a differential equation *exactly*.

> This is not what this module is about, but some special cases will help us to test our methods!

## Example 1

Consider the function $y(t) = t^2$.

1. We can plot the function and it's derivative
2. We can compute $y'(t)$ using the definition of a limit

## Example 2

Consider the function $y(t) = t^3$.

Similarly it is quite easy to compute the derivative!

> In general, it can be shown that when $y(t) = a t^m$, then $y'(t) = a m t^{m-1}$.

## Example 3

By working backwards from a known expression for $y(t)$ and $y'(t)$, we can make up our own differential equation that has $y(t)$ as a known solution.

Consider, for example, $y(t) = t^3$, which has $y'(t) = 3 t^2$.

We see that $y(t) = t^3$, is the solution of
$$
y'(t) = \frac{3 y(t)}{t} \quad\text{subject to the initial condition}\quad y(1) = 1.
$$

So, if we solve this differential equation for $t$ between $1$ and $2$, then we know the exact answer when $t = 2$ is $y(2) = 2^3 = 8$.

# Errors in Euler's method

$$
y'(t) = \frac{3 y(t)}{t} \quad\text{subject to the initial condition}\quad y(1) = 1.
$$

Solve this differential equation using Euler's method and check the error at $t = 2$

In [ ]:
headers = ["n", "dt", "solution", "abs. error", "ratio"]
data = []


def euler(f, n, dt, t0, y0, T):
    t = np.double(t0)
    y = np.double(y0)

    t_, y_ = [t], [y]

    for i in range(n):
        y += dt * f(t, y)
        t += dt

        t_.append(t)
        y_.append(y)

    return t_, y_


t0 = 1.0
y0 = 1.0
T = 2.0


def f(t, y):
    return 3 * y / t


exact = 8.0

old_error = None

for n in [10, 20, 40, 80, 160, 320, 640]:
    dt = (T - t0) / n
    t, y = euler(f, n, dt, t0, y0, T)
    error = abs(y[-1] - exact)
    plt.plot(t, y, label=f"{dt=}")

    if old_error is not None:
        ratio = error / old_error
    else:
        ratio = "---"

    old_error = error

    data.append([n, dt, y[-1], error, ratio])

t_ = np.linspace(1, 2)
plt.plot(t_, t_**3, "k--", label="exact")

plt.xlabel("t")
plt.ylabel("y(t)")
plt.grid(True)
plt.legend()

df_euler = pd.DataFrame(data, columns=headers)

## The errors at $t=2$

In [ ]:
plt.figure(figsize=(8, 5.2))

plt.subplot(1, 2, 1)
plt.semilogx(df_euler["dt"], df_euler["solution"], ".-")
plt.xlabel("$\\mathrm{d}t$")
plt.ylabel("$y(2.0)$")
plt.grid(True)

plt.tight_layout()
plt.show()

## The errors at $t=2$

In [ ]:
plt.figure(figsize=(8, 5.2))

plt.subplot(1, 2, 1)
plt.semilogx(df_euler["dt"], df_euler["solution"], ".-")
plt.xlabel("$\\mathrm{d}t$")
plt.ylabel("$y(2.0)$")
plt.grid(True)

plt.subplot(1, 2, 2)
plt.loglog(df_euler["dt"], df_euler["abs. error"], ".-")
plt.xlabel("$\\mathrm{d}t$")
plt.ylabel("abs. error")
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
df_euler.style.hide().set_caption("Results of using Euler's method varying dt")

> Each time $\mathrm{d}t$ is halved the error is halved!?!?!?!?!?!?!?

What might we expect the computed solution to be if we halved $\mathrm{d}t$ one more time?

## Big O Notation

In considering algorithm complexity, you have already seen big O notation. For example:

-   Gaussian elimination requires $O(n^3)$ operations when $n$ is large.
-   Backward substitution requires $O(n^2)$ operations when $n$ is large.

For large values of $n$, the *highest* powers of $n$ are most significant.

For small values of $\mathrm{d}t$, it is the *lowest* power of $\mathrm{d}t$ that are most significant:

- e.g., when $\mathrm{d}t = 0.001$, $\mathrm{d}t$ is much bigger than $(\mathrm{d}t)^2$

We can make use of the "big O" notation in either case.

- For example, suppose $$
  f(x) = 2 x^2 + 4 x^3 + x^5 + 2 x^6,
  $$

    -   then $f(x) = O(x^6)$ as $x \to \infty$
    -   and $f(x) = O(x^2)$ as $x \to 0$.

-   In this notation, we can say that **the error in Euler's method is $O(\mathrm{d}t)$**.

# Improving on Euler's method

So the error is proportional to $\mathrm{d}t$, .... so what?

## The proposal

1. Take one time step of size $\mathrm{d}t$ - call this solution $\alpha$
2. Take two time steps of size $\frac{1}{2} \mathrm{d}t$ - call this solution $\beta$

The error using $\beta$ should be *about* half the error using $\alpha$:

$$
\begin{aligned}
\alpha - y_{\text{exact}} = E \\
\beta - y_{\text{exact}} = E/2.
\end{aligned}
$$

Rearranging gives:
$$
y_{\text{exact}} = 2 \beta - \alpha
$$
which should be an improve approximation!

We use this approach to construct **the midpoint method**

## The midpoint method

Assume we have already computed up to time step $i$: $t^{(i)}$ and $y^{(i)}$

1. Take one time step of size $\mathrm{d}t$ - call this solution $\alpha$:

    $$
    \begin{aligned}
    \alpha & = y^{(i)} + \mathrm{d}t f(t^{(i)}, y^{(i)}) \\
    t^{(i+1)} & = t^{(i)} + \mathrm{d} t
    \end{aligned}
    $$
    
2. Take two time steps of size $\frac{1}{2} \mathrm{d}t$ - call this solution $\beta$:

    $$
    \begin{aligned}
    k & = y^{(i)} + (\mathrm{d}t / 2) f(t^{(i)}, y^{(i)}) \\
    m & = t^{(i)} + \mathrm{d}t / 2 \\
    \beta & = k + (\mathrm{d}t / 2) f(m, k) \\
    t^{(i+1)} & = m + \mathrm{d}t / 2.
    \end{aligned}
    $$

Combine $\alpha$ and $\beta$ as suggested on the previous slide...

... results in

$$
\begin{aligned}
	k & = y^{(i)} + (\mathrm{d}t / 2) f(t^{(i)}, y^{(i)}) \\
	m & = t^{(i)} + \mathrm{d}t / 2 \\
	y^{(i+1)} & = y^{(i)} + \mathrm{d}t f(m, k) \\
	t^{(i+1)} & = t^{(i)} + \mathrm{d}t.
\end{aligned}
$$

```python
for i in range(n):
    y_mid = y[i] + 0.5 * dt * f(t[i], y[i])
    t_mid = t[i] + 0.5 * dt
    y[i + 1] = y[i] + dt * f(t_mid, y_mid)
    t[i + 1] = t[i] + dt
```

## The results

We look at the results for the same problem as before for the solution at the final time $t = 2.0$:

In [ ]:
headers = ["n", "dt", "solution", "abs. error", "ratio"]
data = []


def midpoint(f, n, dt, t0, y0, T):
    t = np.double(t0)
    y = np.double(y0)

    for i in range(n):
        k = y + (dt / 2) * f(t, y)
        m = t + dt / 2
        y += dt * f(m, k)
        t += dt

    return y


t0 = 1.0
y0 = 1.0
T = 2.0


def f(t, y):
    return 3 * y / t


exact = 8.0

old_error = None

for n in [10, 20, 40, 80, 160, 320, 640]:
    dt = (T - t0) / n
    y = midpoint(f, n, dt, t0, y0, T)
    error = abs(y - exact)

    if old_error is not None:
        ratio = error / old_error
    else:
        ratio = "---"

    old_error = error

    data.append([n, dt, y, error, ratio])

df_midpoint = pd.DataFrame(data, columns=headers)

In [ ]:
plt.figure(figsize=(8, 5.2))

plt.subplot(1, 2, 1)
plt.semilogx(df_euler["dt"], df_euler["solution"], ".-", label="Euler")
plt.semilogx(df_midpoint["dt"], df_midpoint["solution"], ".-", label="Midpoint")

plt.xlabel("$\\mathrm{d}t$")
plt.ylabel("$y(2.0)$")
plt.grid(True)
plt.legend()

plt.subplot(1, 2, 2)
plt.loglog(df_euler["dt"], df_euler["abs. error"], ".-", label="Euler")
plt.xlabel("$\\mathrm{d}t$")
plt.ylabel("abs. error")
plt.grid(True)


plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(8, 5.2))

plt.subplot(1, 2, 1)
plt.semilogx(df_euler["dt"], df_euler["solution"], ".-", label="Euler")
plt.semilogx(df_midpoint["dt"], df_midpoint["solution"], ".-", label="Midpoint")

plt.xlabel("$\\mathrm{d}t$")
plt.ylabel("$y(2.0)$")
plt.grid(True)
plt.legend()

plt.subplot(1, 2, 2)
plt.loglog(df_euler["dt"], df_euler["abs. error"], ".-", label="Euler")
plt.loglog(df_midpoint["dt"], df_midpoint["abs. error"], ".-", label="Midpoint")
plt.xlabel("$\\mathrm{d}t$")
plt.ylabel("abs. error")
plt.grid(True)


plt.tight_layout()
plt.show()

In [ ]:
df_midpoint.style.hide().set_caption("Results of using the midpoint method varying dt")

> Each time $\mathrm{d}t$ is halved the error is quartered!?!?!?!?!?!?!?

## What about the big O?

The error quarters each time that the time interval size $\mathrm{d}t$ is halved - the error is proportional to $(\mathrm{d}t)^2$.

The error of the midpoint method is $O(\mathrm{d}t^2)$ as $\mathrm{d}t \to 0$

This is a significant improvement over Euler's method:

- Euler is a **first order** method
- the midpoint method is a **second order** method

## Example

Take two steps of the midpoint method to approximate the solution of

$$
    y'(t) = y(1-y) \text{ subject to the initial condition } y(0) = 2,
$$

for $0 \le t \le 1$.

# Summary

-   In some special cases exact solutions of differential equations can be found - this is not true in general however.

-   Computational modelling is required for most problems of practical interest (and will of course work just as well even if an exact solution could be found).

-   Comparison with a known solution shows that Euler's method leads to an error that is proportional to $\mathrm{d}t$.

-   The midpoint scheme's error is proportional to $(\mathrm{d}t)^2$ but requires about twice the computational work per step.

-   Only 2 computational schemes are introduced here - there are many more that we don't consider...